In [5]:
# %load ./rbm.py
import torch


class RBM():

    def __init__(self, num_visible, num_hidden, k, learning_rate=1e-3, momentum_coefficient=0.5, weight_decay=1e-4, use_cuda=True):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.k = k
        self.learning_rate = learning_rate
        self.momentum_coefficient = momentum_coefficient
        self.weight_decay = weight_decay
        self.use_cuda = use_cuda

        self.weights = torch.randn(num_visible, num_hidden) * 0.1
        self.visible_bias = torch.ones(num_visible) * 0.5
        self.hidden_bias = torch.zeros(num_hidden)

        self.weights_momentum = torch.zeros(num_visible, num_hidden)
        self.visible_bias_momentum = torch.zeros(num_visible)
        self.hidden_bias_momentum = torch.zeros(num_hidden)

        if self.use_cuda:
            self.weights = self.weights.cuda()
            self.visible_bias = self.visible_bias.cuda()
            self.hidden_bias = self.hidden_bias.cuda()

            self.weights_momentum = self.weights_momentum.cuda()
            self.visible_bias_momentum = self.visible_bias_momentum.cuda()
            self.hidden_bias_momentum = self.hidden_bias_momentum.cuda()

    def sample_hidden(self, visible_probabilities):
        print(visible_probabilities.shape)
        print(self.weights.shape)
        hidden_activations = torch.matmul(visible_probabilities, self.weights) + self.hidden_bias
        hidden_probabilities = self._sigmoid(hidden_activations)
        return hidden_probabilities

    def sample_visible(self, hidden_probabilities):
        visible_activations = torch.matmul(hidden_probabilities, self.weights.t()) + self.visible_bias
        visible_probabilities = self._sigmoid(visible_activations)
        return visible_probabilities

    def contrastive_divergence(self, input_data):
        # Positive phase
        positive_hidden_probabilities = self.sample_hidden(input_data)
        positive_hidden_activations = (positive_hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()
        positive_associations = torch.matmul(input_data.t(), positive_hidden_activations)

        # Negative phase
        hidden_activations = positive_hidden_activations

        for step in range(self.k):
            visible_probabilities = self.sample_visible(hidden_activations)
            hidden_probabilities = self.sample_hidden(visible_probabilities)
            hidden_activations = (hidden_probabilities >= self._random_probabilities(self.num_hidden)).float()

        negative_visible_probabilities = visible_probabilities
        negative_hidden_probabilities = hidden_probabilities

        negative_associations = torch.matmul(negative_visible_probabilities.t(), negative_hidden_probabilities)

        # Update parameters
        self.weights_momentum *= self.momentum_coefficient
        self.weights_momentum += (positive_associations - negative_associations)

        self.visible_bias_momentum *= self.momentum_coefficient
        self.visible_bias_momentum += torch.sum(input_data - negative_visible_probabilities, dim=0)

        self.hidden_bias_momentum *= self.momentum_coefficient
        self.hidden_bias_momentum += torch.sum(positive_hidden_probabilities - negative_hidden_probabilities, dim=0)

        batch_size = input_data.size(0)

        self.weights += self.weights_momentum * self.learning_rate / batch_size
        self.visible_bias += self.visible_bias_momentum * self.learning_rate / batch_size
        self.hidden_bias += self.hidden_bias_momentum * self.learning_rate / batch_size

        self.weights -= self.weights * self.weight_decay  # L2 weight decay

        # Compute reconstruction error
        error = torch.sum((input_data - negative_visible_probabilities)**2)

        return error

    def _sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))

    def _random_probabilities(self, num):
        random_probabilities = torch.rand(num)

        if self.use_cuda:
            random_probabilities = random_probabilities.cuda()

        return random_probabilities




In [6]:
from PIL import Image
#from IPython.display import display, Image

In [7]:
# %load mnist_example.py
import numpy as np
from sklearn.linear_model import LogisticRegression
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms

In [25]:
import redis
import io
import base64

def decoder(img_str):
    return base64.b64encode(img_str).decode('utf-8')

def encoder(img_str):
    return base64.b64decode(img_str.decode('utf-8'))

def fetch_ring_im(ind, redis):
    img_str = redis.get('frame_%05d' % ind)
    enc_str = encoder(img_str)
    img = Image.open(io.BytesIO(enc_str))
    return img

def fetch_tensor_im(ind, redis, t):
    im = fetch_ring_im(ind, redis)
    t = t(im)
    return t.view(3, 720 * 1024)

In [26]:
VISIBLE_UNITS = 720 * 1024
HIDDEN_UNITS = 640
EPOCHS=5
BATCH_SIZE=3
CD_K=2
CUDA=False

In [23]:
from rbm import RBM
rbm = RBM(VISIBLE_UNITS, HIDDEN_UNITS, CD_K, use_cuda=CUDA)
print('RBM Initialized...')

RBM Initialized...


In [ ]:
frame = 0
ringsize = 10000
r = redis.StrictRedis(host='redis')
t = torchvision.transforms.ToTensor()
while True:
    tens = fetch_tensor_im(frame, r, t)
    data_loader = torch.utils.data.DataLoader(tens, batch_size=BATCH_SIZE)
    ########## TRAINING RBM ##########
    print('Training RBM on frame_%05d...' % frame)
    for epoch in range(EPOCHS):
        epoch_error = 0.0

        for batch in data_loader:
            #batch = batch.view(len(batch), VISIBLE_UNITS)  # flatten input data

            if CUDA:
                batch = batch.cuda()

            batch_error = rbm.contrastive_divergence(batch)

            epoch_error += batch_error

        print('Epoch Error (epoch=%d): %.4f' % (epoch, epoch_error))
    frame += 1
    if frame > ringsize:
        frame = 0

Training RBM on frame_00000...
Epoch Error (epoch=0): 894552.2413
Epoch Error (epoch=1): 806899.1297
Epoch Error (epoch=2): 787612.2400
Epoch Error (epoch=3): 787870.9363
Epoch Error (epoch=4): 786755.0162
Training RBM on frame_00001...
Epoch Error (epoch=0): 789381.4138
Epoch Error (epoch=1): 788078.2660
Epoch Error (epoch=2): 787596.7765
Epoch Error (epoch=3): 787272.9111
Epoch Error (epoch=4): 785868.0121
Training RBM on frame_00002...
Epoch Error (epoch=0): 783801.1623
Epoch Error (epoch=1): 782783.4660
Epoch Error (epoch=2): 781876.9494
Epoch Error (epoch=3): 780987.6345
Epoch Error (epoch=4): 780735.8847
Training RBM on frame_00003...
Epoch Error (epoch=0): 779077.3805
Epoch Error (epoch=1): 777555.1780
Epoch Error (epoch=2): 776368.4218
Epoch Error (epoch=3): 776585.7109
Epoch Error (epoch=4): 775043.5334
Training RBM on frame_00004...
Epoch Error (epoch=0): 774303.4899
Epoch Error (epoch=1): 773571.6007
Epoch Error (epoch=2): 772840.1275
Epoch Error (epoch=3): 772360.6784
Epoch

In [ ]:
sample = rbm.sample_visible(torch.randn(3, 640))
sample

In [ ]:
sample = rbm.sample_visible(torch.randn(3, 640))
torchvision.transforms.ToPILImage()(sample.view(3, 720, 1024))

In [ ]:
l = rbm.sample_hidden(simply_flattened)
torchvision.transforms.ToPILImage()(l.view(3, 32, 20))

In [ ]:
v = rbm.sample_visible(l)
torchvision.transforms.ToPILImage()(v.view(3, 720, 1024))

In [ ]:
im

In [ ]:
la = rbm.sample_hidden(simply_flattened)
la_n = la.numpy()
co = np.corrcoef(la_n)

In [ ]:
stamp = torchvision.transforms.ToPILImage()(la.view(3, 32, 20))
stamp.resize([720, 1024])

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
df = pd.DataFrame(la_n)
p = df.T.plot.kde(figsize=(16,8))

In [ ]:
# from scipy import stats
# stats.gaussian_kde?

In [ ]:
# kde = stats.gaussian_kde(la_n, )

In [ ]:
from sklearn.neighbors import KernelDensity
KernelDensity?

In [ ]:
kde = KernelDensity(
    bandwidth=0.000001
).fit(la_n.T)
kde

In [ ]:
# projected_image = kde.resample(640)
projected_image = kde.sample(640).T
pd.DataFrame(projected_image).T.plot.kde(figsize=(16,8))

In [ ]:
la_n.shape == projected_image.shape, np.any(la_n == projected_image)

In [ ]:
projected_image_squashed = projected_image.copy()
projected_image_squashed[projected_image_squashed > 1] = 1
projected_image_squashed[projected_image_squashed < 0] = 0

In [ ]:
proj_t = torch.Tensor(projected_image_squashed)
stamp_e = torchvision.transforms.ToPILImage()(proj_t.view(3, 32, 20))
stamp_e.resize([720, 1024])

In [ ]:

fig, axarr = plt.subplots(1,2, figsize=(10, 10))

axarr[0].imshow(stamp)
axarr[1].imshow(stamp_e)

In [ ]:
vv = rbm.sample_visible(proj_t)
torchvision.transforms.ToPILImage()(vv.view(3, 720, 1024))

In [ ]:
a = torchvision.transforms.ToPILImage()(vv.view(3, 720, 1024))
b = torchvision.transforms.ToPILImage()(v.view(3, 720, 1024))

fig, axarr = plt.subplots(1,2, figsize=(20, 10))

axarr[0].imshow(a)
axarr[1].imshow(b)

In [ ]:
im

In [ ]:
# projected_image = kde.resample(640)
projected_image_sequence = kde.sample(640 * 1440).T
pd.DataFrame(projected_image_sequence).T.plot.kde(figsize=(16,8))

In [ ]:
a = projected_image_sequence[:, 0*640:1*640] 
b = projected_image_sequence[:, 1439*640:1440*640]
a.shape, b.shape
np.any(a == b)

In [ ]:
from tqdm import tnrange, tqdm_notebook
for i in tnrange(1440, desc='frames'):
    p = projected_image_sequence[:, i*640:(i+1)*640]
    p[p > 1] = 1
    p[p < 0] = 0
    proj_t = torch.Tensor(p)
    pp = rbm.sample_visible(proj_t)
    ipp = torchvision.transforms.ToPILImage()(pp.view(3, 720, 1024))
    ipp.save('./image/%05i_projected_shimmer.png' % i)